In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import os
import csv
import pandas as pd
import numpy as np
import requests
import time
import json
from pandas import json_normalize
from scipy.stats import linregress
from pprint import pprint

In [7]:
# Set the API base URL 
url = f"https://data.austintexas.gov/resource/9t4d-g238.json"

# Create URL for GET requests
animal_data = requests.get(url).json()

# Convert the API to a DataFrame and display data
animal_df = pd.DataFrame.from_dict(json_normalize(animal_data))
animal_df.tail(5)

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
995,A722325,Bella,2016-05-16T15:56:00.000,2016-05-16T15:56:00.000,2015-08-14T00:00:00.000,Return to Owner,Dog,Spayed Female,9 months,Labrador Retriever Mix,Black,NaN
996,A664032,Parker,2013-10-01T17:18:00.000,2013-10-01T17:18:00.000,2013-07-27T00:00:00.000,Adoption,Dog,Spayed Female,2 months,German Shepherd Mix,Black/Tan,NaN
997,A785557,Spicy,2018-12-16T14:59:00.000,2018-12-16T14:59:00.000,2016-12-05T00:00:00.000,Return to Owner,Dog,Intact Male,2 years,American Pit Bull Terrier Mix,Brown/White,NaN
998,A800965,Marley,2019-08-17T16:40:00.000,2019-08-17T16:40:00.000,2016-07-29T00:00:00.000,Adoption,Dog,Spayed Female,3 years,Blue Lacy/Beagle,Blue/White,NaN
999,A805280,*Vincent,2019-12-14T16:55:00.000,2019-12-14T16:55:00.000,2019-06-06T00:00:00.000,Adoption,Cat,Neutered Male,6 months,Domestic Shorthair,Orange Tabby/White,NaN


In [8]:
## JSON results are limited and will not provide 2022 information.
## Switching from JSON to CSV.

In [10]:
# Path to collect CSV data from folder
animal_data = os.path.join("Project_1", "Resources", "Animal_Outcome.csv")

# Convert CSV to DataFrame
animal_df = pd.read_csv("C://Users//Junk//Desktop//Project_1//Resources//Animal_Outcome.csv") 
animal_df.tail(5)

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
153437,A883539,*Pipsqueak,07/19/2023 05:18:00 PM,Jul 2023,02/24/2023,Adoption,NaN,Dog,Intact Female,4 months,Labrador Retriever Mix,Cream/White
153438,A884334,Noche,07/19/2023 05:14:00 PM,Jul 2023,04/01/2023,Adoption,NaN,Cat,Spayed Female,3 months,Domestic Shorthair,Black
153439,A885055,Sparky,07/19/2023 05:00:00 PM,Jul 2023,04/12/2023,Adoption,NaN,Cat,Neutered Male,3 months,Siamese,Flame Point
153440,A884031,Bowzer,07/19/2023 05:26:00 PM,Jul 2023,06/27/2019,Adoption,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Cream/Yellow
153441,A883741,*Fiddle,07/19/2023 06:57:00 PM,Jul 2023,06/22/2022,Transfer,Partner,Dog,Intact Male,1 year,Australian Cattle Dog Mix,Tan/White


In [13]:
# Creating a DataFrame copy to reduce to essential columns
revised_df= animal_df[["Animal ID", "MonthYear", "Outcome Type", "Animal Type", "Breed"]].copy()
revised_df.tail(5)

,Animal ID,MonthYear,Outcome Type,Animal Type,Breed
153437,A883539,Jul 2023,Adoption,Dog,Labrador Retriever Mix
153438,A884334,Jul 2023,Adoption,Cat,Domestic Shorthair
153439,A885055,Jul 2023,Adoption,Cat,Siamese
153440,A884031,Jul 2023,Adoption,Dog,Labrador Retriever Mix
153441,A883741,Jul 2023,Transfer,Dog,Australian Cattle Dog Mix


In [15]:
# Rename column and split a DataFrame column
revised_df.rename(columns = {"MonthYear":"Month Year"}, inplace = True)
revised_df[["Month","Year"]] = revised_df["Month Year"].str.split(" ",expand=True)
revised_df.tail(5)

,Animal ID,Month Year,Outcome Type,Animal Type,Breed,Month,Year
153437,A883539,Jul 2023,Adoption,Dog,Labrador Retriever Mix,Jul,2023
153438,A884334,Jul 2023,Adoption,Cat,Domestic Shorthair,Jul,2023
153439,A885055,Jul 2023,Adoption,Cat,Siamese,Jul,2023
153440,A884031,Jul 2023,Adoption,Dog,Labrador Retriever Mix,Jul,2023
153441,A883741,Jul 2023,Transfer,Dog,Australian Cattle Dog Mix,Jul,2023


In [17]:
# Cleaning and sorting the data
split_df = revised_df[["Animal ID", "Month", "Year", "Outcome Type", "Animal Type", "Breed"]].copy()
split_df.tail(5)

,Animal ID,Month,Year,Outcome Type,Animal Type,Breed
153437,A883539,Jul,2023,Adoption,Dog,Labrador Retriever Mix
153438,A884334,Jul,2023,Adoption,Cat,Domestic Shorthair
153439,A885055,Jul,2023,Adoption,Cat,Siamese
153440,A884031,Jul,2023,Adoption,Dog,Labrador Retriever Mix
153441,A883741,Jul,2023,Transfer,Dog,Australian Cattle Dog Mix


In [18]:
# Display number of intakes per year
split_df["Year"].value_counts()

2019    19775
2014    18709
2015    18506
2016    17670
2017    17657
2018    16741
2021    11966
2022    11880
2020     9777
2023     6256
2013     4505
Name: Year, dtype: int64

In [19]:
cleaned_intake_df = split_df[split_df["Year"] == "2022"]
cleaned_intake_df.tail(5)

,Animal ID,Month,Year,Outcome Type,Animal Type,Breed
153073,A857909,Jun,2022,Adoption,Dog,Pit Bull Mix
153108,A853013,Apr,2022,Adoption,Cat,Domestic Shorthair
153197,A862948,Aug,2022,Transfer,Cat,Domestic Shorthair Mix
153355,A850898,Feb,2022,Adoption,Dog,Siberian Husky Mix
153422,A869924,Dec,2022,Transfer,Cat,Domestic Shorthair


In [20]:
final_version_df = cleaned_intake_df.reset_index(drop=True)
final_version_df.tail(5)

,Animal ID,Month,Year,Outcome Type,Animal Type,Breed
11875,A857909,Jun,2022,Adoption,Dog,Pit Bull Mix
11876,A853013,Apr,2022,Adoption,Cat,Domestic Shorthair
11877,A862948,Aug,2022,Transfer,Cat,Domestic Shorthair Mix
11878,A850898,Feb,2022,Adoption,Dog,Siberian Husky Mix
11879,A869924,Dec,2022,Transfer,Cat,Domestic Shorthair


In [21]:
final_version_df["Month"].value_counts()

Aug    1271
Jul    1198
Jun    1198
May    1081
Sep    1043
Jan    1012
Apr     970
Dec     867
Mar     851
Feb     803
Oct     800
Nov     786
Name: Month, dtype: int64

In [22]:
final_version_df["Animal Type"].value_counts()

Dog          6412
Cat          4766
Other         592
Bird          109
Livestock       1
Name: Animal Type, dtype: int64

In [23]:
final_version_df["Outcome Type"].value_counts()

Adoption           6737
Transfer           3097
Return to Owner    1126
Euthanasia          611
Rto-Adopt           155
Died                100
Disposal             48
Missing               3
Stolen                2
Relocate              1
Name: Outcome Type, dtype: int64

In [63]:
#cat_outcomes.DataFrame({"Animal Type": ["Cat"]})
#cat_outcomes_df

#cat_adoptions_df = final_version_df[final_version_df["Animal Type"] == "Cat"]
#cat_adoptions_df.tail(5)



#cat_adoptions_df = final_version_df.set_index("Animal Type")['Outcome Type']
#cat_adoptions_df

Cat = ["Cat"] 
cat_outcomes_df = final_version_df[final_version_df["Animal Type"].isin(Cat)]
cat_outcomes_df.head(5)

,Animal ID,Month,Year,Outcome Type,Animal Type,Breed
0,A860161,Jul,2022,Adoption,Cat,Domestic Shorthair
2,A848454,Jan,2022,Adoption,Cat,Domestic Shorthair Mix
3,A822928,Jan,2022,Adoption,Cat,Domestic Shorthair
5,A852822,Mar,2022,Adoption,Cat,Domestic Shorthair
13,A856357,Jun,2022,Adoption,Cat,Domestic Shorthair


In [59]:


#clean_cat = cat_outcomes_df.set_index("Outcome Type")
#top_cat_outcome = clean_cat.drop(["Euthanasia", "Died", "Missing", "Relocate", "Disposal", "Missing"])
#top_cat_outcome.head

<bound method NDFrame.head of              Animal ID Month  Year Animal Type                   Breed
Outcome Type                                                          
Adoption       A860161   Jul  2022         Cat      Domestic Shorthair
Adoption       A848454   Jan  2022         Cat  Domestic Shorthair Mix
Adoption       A822928   Jan  2022         Cat      Domestic Shorthair
Adoption       A852822   Mar  2022         Cat      Domestic Shorthair
Adoption       A856357   Jun  2022         Cat      Domestic Shorthair
...                ...   ...   ...         ...                     ...
Adoption       A854970   Apr  2022         Cat  Domestic Shorthair Mix
Adoption       A863788   Aug  2022         Cat      Domestic Shorthair
Adoption       A853013   Apr  2022         Cat      Domestic Shorthair
Transfer       A862948   Aug  2022         Cat  Domestic Shorthair Mix
Transfer       A869924   Dec  2022         Cat      Domestic Shorthair

[4554 rows x 5 columns]>

In [68]:
clean_cat = cat_outcomes_df["Outcome Type"].value_counts()
clean_cat 

Adoption           3194
Transfer           1177
Return to Owner     152
Euthanasia          142
Died                 54
Rto-Adopt            31
Disposal             12
Missing               3
Relocate              1
Name: Outcome Type, dtype: int64

In [46]:


Dog = ["Dog"] 
dog_adoptions_df = final_version_df[final_version_df["Animal Type"].isin(Dog)]
dog_adoptions_df

,Animal ID,Month,Year,Outcome Type,Animal Type,Breed
1,A595061,Aug,2022,Return to Owner,Dog,Labrador Retriever Mix
4,A856853,May,2022,Adoption,Dog,Black Mouth Cur Mix
6,A851472,Feb,2022,Transfer,Dog,Cairn Terrier Mix
7,A870443,Dec,2022,Adoption,Dog,Jack Russell Terrier/Chihuahua Shorthair
8,A851434,Feb,2022,Transfer,Dog,Chihuahua Shorthair
...,...,...,...,...,...,...
11872,A862821,Oct,2022,Adoption,Dog,Plott Hound
11873,A854340,Apr,2022,Adoption,Dog,Chihuahua Shorthair
11874,A857909,Nov,2022,Adoption,Dog,Pit Bull Mix
11875,A857909,Jun,2022,Adoption,Dog,Pit Bull Mix


In [ ]:
x_values = final_version_df["Animal Type"]
y_values = final_version_df["Max Temp"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(-40,40),fontsize=15,color="red")
plt.xlabel("Latitude")
plt.ylabel("Max Temp")
plt.xlim(-60, 5)
plt.ylim(0, 42)

print(f"The r-value is: {rvalue}")
plt.show()